# Diagnoza raka piersi

# Import bibliotek

In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from statsmodels.discrete.discrete_model import Logit

# Import danych

In [2]:
data = pd.read_excel("https://archive.ics.uci.edu/ml/machine-learning-databases/00451/dataR2.xlsx")

#Losowy podzial danych na zbiór treningowy (80%) i testowy (20%) 
train, test = model_selection.train_test_split(data,test_size = .20)

#Podzial danych na zmienne zalezne i niezalezne
xtrain = train.drop(columns=['Classification']).values
ytrain = (train["Classification"].values)-1
xtest = test.drop(columns=['Classification']).values
ytest = (test["Classification"].values)-1

# Uczenie modelu

In [3]:
model = Logit(ytrain, xtrain).fit()
ypred = model.predict(xtest)
cut_off = 0.5

Optimization terminated successfully.
         Current function value: 0.487707
         Iterations 8


# Wyliczanie trafnosci i czulosci dla progu odciecia 0.5

In [4]:
#Macierz bledu dla progu odciecia 0,5
tn, fp, fn, tp = confusion_matrix(ytest,ypred >= cut_off).ravel()

#Obliczanie trafnosci z macierzy bledu
accuracy=((tp + tn)/(tn + fp + fn + tp))
print ('Trafnosc dla punktu odciecia (0,5):', accuracy)

#Obliczanie czulosci z macierzy bledu
sensitivity = tp / (tp + fn)
print('Czulosc dla punktu odciecia (0,5):', sensitivity )

Trafnosc dla punktu odciecia (0,5): 0.7083333333333334
Czulosc dla punktu odciecia (0,5): 0.8461538461538461


# Definiowanie kosztów leczenia

In [5]:
tp_koszt = 48477
tn_koszt = 0
fp_koszt = 751
fn_koszt = 89463

# Testowanie nowych progow odciecia przy okreslonych kosztach leczenia
Funkcja wyliczajaca najnizsze koszty leczenia oraz odpowiadajace im punkty odciecia  
Dodatkowa indeksacia zostala dodana po to aby unkinac progu odciecia rownego 0, który  
czasami powodował błedy w dalszej części zdania

In [6]:
punkty_odciecia = np.arange(0, 1.01, 0.01)
wyniki = {}
for punkt in punkty_odciecia:
    tn2, fp2, fn2, tp2 = confusion_matrix(ytest,ypred > punkt).ravel()
    koszt = fn2 * fn_koszt + fp2 * fp_koszt + tn2 * tn_koszt +tp2 * tp_koszt
    wyniki[punkt] = koszt

wyniki = pd.Series(wyniki)
print(wyniki)
liczba_argumentow = wyniki.value_counts()[wyniki.iloc[wyniki.argmin()]]
najnizszy_koszt = wyniki.iloc[wyniki.argmin()]
najlepszy_punkt_odciecia = wyniki.iloc[wyniki.argmin():wyniki.argmin()+liczba_argumentow].index[liczba_argumentow-1]
print('Najnizszy koszt:',najnizszy_koszt)
print('Punkt odciecia dla tego kosztu:',najlepszy_punkt_odciecia)

0.00     638462
0.01     638462
0.02     638462
0.03     637711
0.04     637711
         ...   
0.96     999075
0.97    1122033
0.98    1122033
0.99    1122033
1.00    1163019
Length: 101, dtype: int64
Najnizszy koszt: 636960
Punkt odciecia dla tego kosztu: 0.23


# Funkcja zwracajaca ostateczne wyniki po wprowadzeniu wybranego progu odciecia

In [7]:
def podsumowanie_modelu(punkt_odciecia):
    tn, fp, fn, tp = confusion_matrix(ytest, ypred > punkt_odciecia).ravel()
    specyficznosc = tn / (tn + fp)
    sensitivity = tp / (tp + fn)
    accuracy = (tp + tn)/ (tn + tp + fn + fp)
    print('Raport klasyfikacji:\n{}'.format(classification_report(ytest, ypred > punkt_odciecia)))
    macierz_pomylek = pd.DataFrame(confusion_matrix(ytest, ypred > punkt_odciecia), 
             columns = ['predicted negatives', 'predicted positives'], 
             index = ['actual negatives', 'actual positives'])
    print('\nMacierz pomyłek:\n{}'.format(macierz_pomylek))
    print ('Trafnosc dla optymalnego punktu odciecia:', accuracy)
    print('Czulosc dla optymalnego punktu odciecia:', sensitivity)

# Podsumowanie modelu

In [8]:
podsumowanie_modelu(najlepszy_punkt_odciecia)

Raport klasyfikacji:
              precision    recall  f1-score   support

           0       1.00      0.18      0.31        11
           1       0.59      1.00      0.74        13

    accuracy                           0.62        24
   macro avg       0.80      0.59      0.53        24
weighted avg       0.78      0.62      0.54        24


Macierz pomyłek:
                  predicted negatives  predicted positives
actual negatives                    2                    9
actual positives                    0                   13
Trafnosc dla optymalnego punktu odciecia: 0.625
Czulosc dla optymalnego punktu odciecia: 1.0


Wyniki koncowe są bardzo mocno zależne od tego jakie dane znajdą się w zbiorze testowym.
Jednak niezaleznie od podzialu danych w kazdym przypadku otrzymamy czulosc rowna 1